### import & seed

In [11]:
# import & seed 고정
import os, time, random
import numpy as np
import pandas as pd

import cv2
import timm
import torch
import torch.nn as nn

from PIL import Image
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import StratifiedKFold

import albumentations as A
from albumentations.pytorch import ToTensorV2

from torch.utils.data import Dataset, DataLoader, Subset
from torch.cuda.amp import autocast, GradScaler
from torch.optim.lr_scheduler import CosineAnnealingLR

# 시드 고정
SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.benchmark = True  # 속도 우선
# torch.use_deterministic_algorithms(False)  # 재현성 우선이면 True로


### Settings (path/hyper parameters/device)

In [12]:
# Settings(path/hyper parameters/device)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

data_path = '../data'
model_name = 'tf_efficientnet_b6.aa_in1k'  # safe_create_model이 알아서 폴백 처리

NUM_CLASSES = 17
img_size = 528        # 224, 456, 528 등
LR = 1e-3
EPOCHS = 12
BATCH_SIZE = 16
num_workers = 32      # 환경에 따라 8~16 권장
N_REPEAT = 4          # 같은 원본을 한 에폭에 3번 노출(서로 다른 증강 조합)


### dataset & repeat augmentation

In [13]:
# dataset & repeat augmentation
class ImageDataset(Dataset):
    """CSV: (filename, target) 형식 가정. 테스트에서는 target 칼럼이 dummy여도 OK."""
    def __init__(self, csv_path, img_dir, transform=None):
        self.df = pd.read_csv(csv_path).values  # [[name, target], ...]
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        name, target = self.df[idx]
        img = np.array(Image.open(os.path.join(self.img_dir, name)))
        if self.transform is not None:
            img = self.transform(image=img)['image']
        return img, target

class RepeatAugDataset(Dataset):
    """같은 샘플을 repeats번 노출(호출마다 transform이 달라지므로 서로 다른 증강 조합으로 학습)."""
    def __init__(self, base_dataset, repeats: int = 3):
        self.base = base_dataset
        self.repeats = repeats

    def __len__(self):
        return len(self.base) * self.repeats

    def __getitem__(self, idx):
        base_idx = idx // self.repeats
        return self.base[base_idx]


### Mixup

In [14]:
def mixup_data(x, y, alpha=0.4):
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1.0
    batch_size = x.size(0)
    index = torch.randperm(batch_size).to(x.device)
    mixed_x = lam * x + (1-lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam

def mixup_criterion(criterion, pred, y_a, y_b, lam):
    return lam * criterion(pred, y_a) + (1-lam) * criterion(pred, y_b)

### transforms

In [ ]:
# ===== Toggle here =====
N_SOME = 4  # ← A/B 테스트 시 3 또는 4로만 바꾸면 됩니다.

heavy_one = A.OneOf([  # 헤비급: 최대 1개만 선택
    A.Perspective(scale=(0.05, 0.12), keep_size=True,
                  pad_mode=cv2.BORDER_CONSTANT, pad_val=255),
    A.Affine(translate_percent=(0.0, 0.08), scale=(0.95, 1.05),
             shear=(-12, 12), cval=255, mode=cv2.BORDER_CONSTANT),
    A.CoarseDropout(min_holes=1, max_holes=4,
                    min_height=int(img_size*0.08), max_height=int(img_size*0.30),
                    min_width=int(img_size*0.15),  max_width=int(img_size*0.40),
                    fill_value=0),
    A.CoarseDropout(min_holes=1, max_holes=4,
                    min_height=int(img_size*0.08), max_height=int(img_size*0.30),
                    min_width=int(img_size*0.15),  max_width=int(img_size*0.40),
                    fill_value=255),
    A.MotionBlur(blur_limit=(7, 11), allow_shifted=True),
], p=1.0)

light_pool = [
    # 기존 변형들(선택되면 항상 적용되도록 p=1.0; 적용 개수는 SomeOf의 n으로 제어)
    A.RandomRotate90(p=1.0),
    A.Rotate(limit=180, border_mode=cv2.BORDER_CONSTANT, value=255, p=1.0),
    A.HorizontalFlip(p=1.0),
    A.VerticalFlip(p=1.0),
    A.RandomBrightnessContrast(0.12, 0.12, p=1.0),
    A.Compose([
        A.LongestMaxSize(max_size=int(img_size*1.15), interpolation=cv2.INTER_CUBIC),
        A.PadIfNeeded(int(img_size*1.15), int(img_size*1.15),
                      border_mode=cv2.BORDER_CONSTANT, value=255, p=1.0),
        A.RandomResizedCrop(height=img_size, width=img_size,
                            scale=(0.85, 1.0), ratio=(0.9, 1.1), p=1.0),
    ]),
    A.Downscale(scale_min=0.85, scale_max=0.96, p=1.0),
    A.GaussianBlur(blur_limit=(3, 7), p=1.0),
    A.ImageCompression(quality_lower=40, quality_upper=85, p=1.0),
    A.GaussNoise(var_limit=(5.0, 20.0), p=1.0),

    # 컬러/톤/채널 변형(실물 데이터의 색 캐스트 대응)
    A.HueSaturationValue(hue_shift_limit=12, sat_shift_limit=25, val_shift_limit=15, p=1.0),
    A.RandomGamma(gamma_limit=(60, 140), p=1.0),
    A.RGBShift(r_shift_limit=20, g_shift_limit=20, b_shift_limit=20, p=1.0),
    A.CLAHE(clip_limit=(1, 3), tile_grid_size=(8, 8), p=1.0),
    A.ToGray(p=1.0),
    A.InvertImg(p=1.0),
]

train_transforms = A.Compose([
    A.SomeOf(
        [heavy_one] + light_pool,  # 헤비 1개까지 + 라이트 풀에서 선택
        n=N_SOME,
        p=1.0
    ),
    A.Resize(img_size, img_size, interpolation=cv2.INTER_CUBIC),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2(),
])

test_transforms = A.Compose([
    A.LongestMaxSize(max_size=img_size, interpolation=cv2.INTER_CUBIC),
    A.PadIfNeeded(img_size, img_size, border_mode=cv2.BORDER_CONSTANT, value=255),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2(),
])

/opt/conda/lib/python3.10/site-packages/albumentations/augmentations/transforms.py:1692: UserWarning: Using default interpolation INTER_NEAREST, which is sub-optimal.Please specify interpolation mode for downscale and upscale explicitly.For additional information see this PR https://github.com/albumentations-team/albumentations/pull/584
  warnings.warn(


### data load & train/val split -> dataloader

In [16]:
# [셀 5-K] K-Fold 분할 & DataLoader 준비 (무TTA)
FOLDS = 5

# 전체 train 로드 (변경 없음)
train_dataset_full = ImageDataset(
    os.path.join(data_path, "train.csv"),
    os.path.join(data_path, "train"),
    transform=train_transforms
)

# 테스트 로더는 폴드와 무관하므로 여기서 한 번만 만들어 둡니다.
test_dataset = ImageDataset(
    os.path.join(data_path, "sample_submission.csv"),
    os.path.join(data_path, "test"),
    transform=test_transforms
)
test_dataloader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=num_workers,
    pin_memory=True,
    drop_last=False
)

# 폴드 분할에서 사용될 라벨
y_all = train_dataset_full.df[:, 1].astype(int)

# StratifiedKFold 객체 (셔플 권장)
skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=SEED)

# 이후 셀 8에서 폴드 루프를 돌릴 것이므로 여기서는 로더를 만들지 않습니다.
# 각 fold마다 train/val Subset과 DataLoader는 셀 8에서 생성합니다.
len(train_dataset_full), len(test_dataloader)


(1570, 197)

### model

In [17]:
# model
def safe_create_model(model_name: str, num_classes: int, device):
    tried = []
    def _try(name, **kw):
        tried.append(name)
        return timm.create_model(name, pretrained=True, num_classes=num_classes, **kw).to(device)

    try:
        return _try(model_name, drop_rate=0.3, drop_path_rate=0.1)
    except TypeError:
        try:
            return _try(model_name, drop_rate=0.3)
        except Exception as e:
            last_err = e

    fallbacks = [
        model_name.replace('.', '_'),
        'tf_efficientnet_b6_ns',
        'tf_efficientnet_b6',
        'tf_efficientnet_b4_ns',
        'convnext_tiny',
    ]
    for fb in fallbacks:
        try:
            return _try(fb, drop_rate=0.3)
        except Exception:
            continue

    raise RuntimeError(f"create_model 실패. 시도: {tried} | last_err: {last_err}")


### train/valid roof definition

In [18]:
# train/valid roof definition
scaler = GradScaler()

def train_one_epoch(loader, model, optimizer, loss_fn, device, scheduler=None, mixup_alpha=0.4):
    model.train()
    train_loss = 0.0
    preds_list, targets_list = [], []

    pbar = tqdm(loader)
    for images, targets in pbar:
        images = images.to(device, non_blocking=True)
        targets = targets.to(device, non_blocking=True)

        optimizer.zero_grad(set_to_none=True)
        with autocast(enabled=(device.type == "cuda")):
            if mixup_alpha > 0:
                inputs, targets_a, targets_b, lam = mixup_data(images, targets, mixup_alpha)
                logits = model(inputs)
                loss = mixup_criterion(loss_fn, logits, targets_a, targets_b, lam)
            else:
                logits = model(images)
                loss = loss_fn(logits, targets)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        # ✅ 배치 단위로 LR 스케줄 진행
        if scheduler is not None:
            scheduler.step()

        # --- 메트릭 누적 ---
        train_loss += loss.item()
        preds_list.extend(logits.argmax(1).detach().cpu().numpy())
        targets_list.extend(targets.detach().cpu().numpy())

        # 진행바에 현재 LR 표시(선택)
        if scheduler is not None:
            # optimizer에 그룹이 여러 개면 첫 그룹 기준
            curr_lr = optimizer.param_groups[0]["lr"]
            pbar.set_description(f"Loss: {loss.item():.4f} | LR: {curr_lr:.2e}")
        else:
            pbar.set_description(f"Loss: {loss.item():.4f}")

    # 평균(배치 평균). 샘플 가중 평균을 원하면 batch_size로 가중하세요.
    train_loss /= len(loader)
    train_acc = accuracy_score(targets_list, preds_list)
    train_f1  = f1_score(targets_list, preds_list, average="macro")

    return {"train_loss": train_loss, "train_acc": train_acc, "train_f1": train_f1}


@torch.no_grad()
def validate_one_epoch(loader, model, loss_fn, device):
    model.eval()
    val_loss = 0.0
    preds_list, targets_list = [], []

    for images, targets in tqdm(loader):
        images = images.to(device)
        targets = targets.to(device)
        with autocast():
            logits = model(images)
            loss = loss_fn(logits, targets)

        val_loss += loss.item()
        preds_list.extend(logits.argmax(1).cpu().numpy())
        targets_list.extend(targets.cpu().numpy())

    val_loss /= len(loader)
    val_acc = accuracy_score(targets_list, preds_list)
    val_f1  = f1_score(targets_list, preds_list, average='macro')
    return {"val_loss": val_loss, "val_acc": val_acc, "val_f1": val_f1}


### model/loss/optim/scheduler & train loof + checkpoint

In [19]:
# model/loss/optim/scheduler & train loof + checkpoint
model = safe_create_model(model_name, NUM_CLASSES, device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=1e-4)
scheduler = CosineAnnealingLR(optimizer, T_max=EPOCHS, eta_min=1e-6)

version = 'v1'
best_val_f1 = 0.0
patience, bad = 4, 0

best_ckpts = []  # 폴드별 최고 성능 모델 경로 저장

for fold, (tr_idx, va_idx) in enumerate(skf.split(np.arange(len(train_dataset_full)), y_all)):
    print(f"\n========== Fold {fold} / {FOLDS-1} ==========")

    # --- 폴드별 Subset
    tr_subset = Subset(train_dataset_full, tr_idx)
    va_subset = Subset(train_dataset_full, va_idx)

    # --- 학습만 반복 노출 (같은 원본을 서로 다른 증강 조합으로 N회 보이기)
    tr_repeat = RepeatAugDataset(tr_subset, repeats=N_REPEAT)

    # --- DataLoader
    train_loader = DataLoader(
        tr_repeat,
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=num_workers,
        pin_memory=True,
        drop_last=True
    )
    val_loader = DataLoader(
        va_subset,
        batch_size=BATCH_SIZE,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=True,
        drop_last=False
    )

    # --- 모델/옵티마/스케줄러: 폴드마다 새로 초기화
    model = safe_create_model(model_name, NUM_CLASSES, device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=1e-4)
    total_steps = EPOCHS * len(train_loader)
    scheduler = CosineAnnealingLR(optimizer, T_max=total_steps, eta_min=1e-6)

    best_val_f1 = -1.0
    patience, bad = 4, 0
    ckpt_path = f"../results/checkpoints/{model_name}_{version}/{model_name}_{version}_fold{fold}.pth"
    os.makedirs(f"../results/checkpoints/{model_name}_{version}", exist_ok=True)

    for epoch in range(EPOCHS):
        print(f"\n[Fold {fold}] Epoch {epoch+1}/{EPOCHS}")
        start = time.time()

        tr = train_one_epoch(train_loader, model, optimizer, loss_fn, device, scheduler, mixup_alpha=0.5)
        va = validate_one_epoch(val_loader,   model, loss_fn, device)

        elapsed = time.time() - start
        print(f"Train F1: {tr['train_f1']:.4f} | Val F1: {va['val_f1']:.4f} | "
              f"Train Loss: {tr['train_loss']:.4f} | Val Loss: {va['val_loss']:.4f} | "
              f"Elapsed: {elapsed:.2f}s")

        if va['val_f1'] > best_val_f1:
            best_val_f1 = va['val_f1']
            torch.save({
                "state_dict": model.state_dict(),
                "epoch": epoch + 1,
                "val_f1": best_val_f1,
                "fold": fold,
            }, ckpt_path)
            print("✅ Best (this fold) saved!")
            bad = 0
        else:
            bad += 1
            if bad >= patience:
                print("⏹ Early stopping (this fold).")
                break

    best_ckpts.append(ckpt_path)

print("\nFold best checkpoints:", best_ckpts)


========== Fold 0 / 4 ==========

[Fold 0] Epoch 1/12


Loss: 0.6214 | LR: 9.83e-04: 100%|██████████| 314/314 [01:34<00:00,  3.33it/s]
100%|██████████| 20/20 [00:03<00:00,  5.15it/s]


Train F1: 0.4040 | Val F1: 0.8759 | Train Loss: 1.2711 | Val Loss: 0.3121 | Elapsed: 98.12s
✅ Best (this fold) saved!

[Fold 0] Epoch 2/12


Loss: 1.5318 | LR: 9.33e-04: 100%|██████████| 314/314 [01:39<00:00,  3.16it/s]
100%|██████████| 20/20 [00:03<00:00,  5.14it/s]


Train F1: 0.4957 | Val F1: 0.9056 | Train Loss: 1.0043 | Val Loss: 0.3019 | Elapsed: 103.37s
✅ Best (this fold) saved!

[Fold 0] Epoch 3/12


...

Train F1: 0.5470 | Val F1: 0.9590 | Train Loss: 0.5605 | Val Loss: 0.1499 | Elapsed: 106.33s

[Fold 4] Epoch 11/12


Loss: 0.2279 | LR: 1.80e-05: 100%|██████████| 314/314 [01:41<00:00,  3.09it/s]
100%|██████████| 20/20 [00:03<00:00,  5.11it/s]


Train F1: 0.5508 | Val F1: 0.9672 | Train Loss: 0.5515 | Val Loss: 0.1636 | Elapsed: 105.69s
✅ Best (this fold) saved!

[Fold 4] Epoch 12/12


Loss: 0.3313 | LR: 1.00e-06: 100%|██████████| 314/314 [01:41<00:00,  3.10it/s]
100%|██████████| 20/20 [00:03<00:00,  5.23it/s]

Train F1: 0.5395 | Val F1: 0.9543 | Train Loss: 0.5381 | Val Loss: 0.1839 | Elapsed: 105.07s

Fold best checkpoints: ['../results/checkpoints/tf_efficientnet_b6.aa_in1k_v1/tf_efficientnet_b6.aa_in1k_v1_fold0.pth', '../results/checkpoints/tf_efficientnet_b6.aa_in1k_v1/tf_efficientnet_b6.aa_in1k_v1_fold1.pth', '../results/checkpoints/tf_efficientnet_b6.aa_in1k_v1/tf_efficientnet_b6.aa_in1k_v1_fold2.pth', '../results/checkpoints/tf_efficientnet_b6.aa_in1k_v1/tf_efficientnet_b6.aa_in1k_v1_fold3.pth', '../results/checkpoints/tf_efficientnet_b6.aa_in1k_v1/tf_efficientnet_b6.aa_in1k_v1_fold4.pth']


### inference & save submission file

In [20]:
# ========== inference & save submission file (K-Fold probability averaging, no TTA) ==========

all_probs = []

with torch.no_grad():
    for ckpt_path in best_ckpts:
        # 폴드별 베스트 체크포인트 로드
        state = torch.load(ckpt_path, map_location=device)
        model = safe_create_model(model_name, NUM_CLASSES, device)
        model.load_state_dict(state["state_dict"])
        model.eval()

        fold_probs = []
        for imgs, _ in tqdm(test_dataloader, desc=f"Infer(Test) - {os.path.basename(ckpt_path)}"):
            imgs = imgs.to(device, non_blocking=True)
            with autocast(enabled=(device.type == 'cuda')):
                logits = model(imgs)                  # (N, C) on cuda (half일 수 있음)
                probs  = logits.float().softmax(1)   # ★ GPU에서 FP32 softmax
            fold_probs.append(probs.cpu())           # (N, C) on cpu float32

        # (T, C) 이 폴드의 전체 확률
        all_probs.append(torch.cat(fold_probs, dim=0))

# (F, T, C) → 확률 평균 → (T,)
probs_mean = torch.stack(all_probs, dim=0).mean(0)   # cpu float32
preds = probs_mean.argmax(1).numpy()

# 제출 파일 저장
sub = pd.read_csv(os.path.join(data_path, "sample_submission.csv"))
sub["target"] = preds[: len(sub)]
out_path = f"../results/submission_{model_name}_{version}.csv"
sub.to_csv(out_path, index=False)
print(f"📄 Saved submission: {out_path}")


Infer(Test) - tf_efficientnet_b6.aa_in1k_v1_fold0.pth: 100%|██████████| 197/197 [00:19<00:00, 10.22it/s]
Infer(Test) - tf_efficientnet_b6.aa_in1k_v1_fold1.pth: 100%|██████████| 197/197 [00:19<00:00, 10.31it/s]
Infer(Test) - tf_efficientnet_b6.aa_in1k_v1_fold2.pth: 100%|██████████| 197/197 [00:19<00:00, 10.15it/s]
Infer(Test) - tf_efficientnet_b6.aa_in1k_v1_fold3.pth: 100%|██████████| 197/197 [00:19<00:00, 10.26it/s]
Infer(Test) - tf_efficientnet_b6.aa_in1k_v1_fold4.pth: 100%|██████████| 197/197 [00:19<00:00, 10.16it/s]

📄 Saved submission: ../results/submission_tf_efficientnet_b6.aa_in1k_v1.csv
